<a href="https://colab.research.google.com/github/AlejoGomezQ/Practica_04_Analitica_Datos/blob/main/optimizacion_arbol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimización del arbol de decisión con GridSearchCV



## 1. Importar librerías

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## 2. Cargar y preparar datos

In [ ]:
df = pd.read_csv('Registro_vendedores_limpio.csv')

posibles_nombres_estado = ['ESTADO', 'Estado', 'estado']
columna_estado = None

for nombre in posibles_nombres_estado:
    if nombre in df.columns:
        columna_estado = nombre
        break

y = df[columna_estado]
X = df.drop(columns=[columna_estado])

print(f"Dimensiones: {X.shape}")
print(f"Variable objetivo: {columna_estado}")

## 3. Codificación de variables

In [ ]:
le_X = {}
X_encoded = X.copy()

for col in X.columns:
    if X[col].dtype == 'object':
        le_X[col] = LabelEncoder()
        X_encoded[col] = le_X[col].fit_transform(X[col].astype(str))

le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y)

X_encoded.head()

## 4. División de datos (70% entrenamiento, 30% prueba)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
)

print(f"Entrenamiento: {X_train.shape[0]} registros")
print(f"Prueba: {X_test.shape[0]} registros")

## 6. Definición del Grid de Hiperparámetros

- **criterion**: Función para medir la calidad de la división ('gini', 'entropy')
- **max_depth**: Profundidad máxima del árbol (None, 5, 10, 15, 20)
- **min_samples_split**: Mínimo de muestras requeridas para dividir un nodo (2, 5, 10, 20)
- **min_samples_leaf**: Mínimo de muestras requeridas en un nodo hoja (1, 2, 4, 8)
- **max_features**: Número máximo de características a considerar (None, 'sqrt', 'log2')

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

print(f"Total de combinaciones a probar: {2 * 5 * 4 * 4 * 3} = {2 * 5 * 4 * 4 * 3}")

## 7. GridSearchCV - Búsqueda de mejores hiperparámetros

Se utiliza validación cruzada de 5 folds (cv=5) para evaluar cada combinación de hiperparámetros.

In [ ]:
grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=1
)

print("Iniciando búsqueda de hiperparámetros...")
grid_search.fit(X_train, y_train)
print("Búsqueda completada")

## 8. Mejores hiperparámetros encontrados

In [ ]:
print("MEJORES HIPERPARÁMETROS")
print("="*60)
for param, valor in grid_search.best_params_.items():
    print(f"{param}: {valor}")

print(f"\nMejor F1-Score en validación cruzada: {grid_search.best_score_:.4f}")

## 9. Evaluación del Modelo Optimizado

In [ ]:
modelo_optimizado = grid_search.best_estimator_
y_pred_optimizado = modelo_optimizado.predict(X_test)

print("MODELO OPTIMIZADO - CON GRIDSEARCHCV")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_optimizado):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_optimizado, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_optimizado, average='weighted'):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_optimizado, average='weighted'):.4f}")
print(f"\nMatriz de Confusión:\n{confusion_matrix(y_test, y_pred_optimizado)}")

## 11. Visualización del Árbol Optimizado

In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(
    modelo_optimizado,
    feature_names=X.columns,
    class_names=le_y.classes_,
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title("Árbol de Decisión Optimizado", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()